In [2]:
import os
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
import time 
from pymongo import MongoClient
import csv

In [3]:
# Make a reference to thecsv file path
csv_path = "MentalHealthCSV.csv"
csv_path2="Male.csv"
csv_path3="Female.csv"

# Import the books.csv file as a DataFrame
MentalHealth_df_c = pd.read_csv(csv_path)
Males = pd.read_csv(csv_path2)
Females = pd.read_csv(csv_path3)
Females= Females.rename(columns={
    "Bad Days Count	": "Bad Days Count",
    "Bad Days Q2	": "Bad Days Q2", 
    
    })

Females.head()


,Sex,Location,Bad Days Count,Bad Days Mean,Bad Days Min,Bad Days Q1,Bad Days Q2,Bad Days Q3,Bad Days Max
0,Female,Alabama,1,6.1,6.1,6.1,6.1,6.1,6.1
1,Female,Alaska,1,4.6,4.6,4.6,4.6,4.6,4.6
2,Female,Arizona,1,4.9,4.9,4.9,4.9,4.9,4.9
3,Female,Arkansas,1,6.5,6.5,6.5,6.5,6.5,6.5
4,Female,California,1,4.2,4.2,4.2,4.2,4.2,4.2


In [4]:
#Count for rosw missing data 
MentalHealth_df_c.count()

*Are you self-employed?*                                                             352
How many employees does your company or organization have?                           304
Is your employer primarily a tech company/organization?                              304
Is your primary role within your company related to tech/IT?                         304
Does your employer provide mental health benefits as part of healthcare coverage?    304
                                                                                    ... 
What country do you *live* in?                                                       352
What US state or territory do you *live* in?                                         204
What is your race?                                                                   204
What country do you *work* in?                                                       352
What US state or territory do you *work* in?                                         204
Length: 82, dtype: in

In [5]:
MentalHealth_df_c.isna().head()

#all of them 

,*Are you self-employed?*,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",...,Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.,"If there is anything else you would like to tell us that has not been covered by the survey questions, please use this space to do so.",Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),What is your age?,What is your gender?,What country do you *live* in?,What US state or territory do you *live* in?,What is your race?,What country do you *work* in?,What US state or territory do you *work* in?
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False


In [6]:
MH_Clean = MentalHealth_df_c.rename(columns={
    "What is your age?": "Age",
    "What is your gender?": "Sex", 
    "What US state or territory do you *live* in?": "Location",
    "Does your employer provide mental health benefits as part of healthcare coverage?": "Emp. Coverage",
    "Overall, how well do you think the tech industry supports employees with mental health issues?": "Ranking",
    "Do you *currently* have a mental health disorder?": "Do you *currently* have a mental health disorder?",
    
    })
MH_Clean.head()

,*Are you self-employed?*,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Emp. Coverage,Do you know the options for mental health care available under your employer-provided health coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health disorders and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",...,Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.,"If there is anything else you would like to tell us that has not been covered by the survey questions, please use this space to do so.",Would you be willing to talk to one of us more extensively about your experiences with mental health issues in the tech industry? (Note that all interview responses would be used _anonymously_ and only with your permission.),Age,Sex,What country do you *live* in?,Location,What is your race?,What country do you *work* in?,What US state or territory do you *work* in?
0,False,26-100,True,True,I don't know,No,Yes,Yes,I don't know,Very easy,...,NaN,NaN,False,25,Male,United States of America,Nebraska,White,United States of America,Nebraska
1,False,26-100,True,True,Yes,No,No,Yes,Yes,I don't know,...,NaN,NaN,False,51,male,United States of America,Nebraska,White,United States of America,Nebraska
2,False,26-100,True,True,I don't know,No,No,I don't know,I don't know,Somewhat difficult,...,I think opening up more conversation around th...,Thank you,True,27,Male,United States of America,Illinois,White,United States of America,Illinois
3,False,100-500,True,True,I don't know,No,Yes,Yes,Yes,Very easy,...,NaN,NaN,False,37,male,United States of America,Nebraska,White,United States of America,Nebraska
4,False,26-100,True,True,I don't know,No,I don't know,I don't know,I don't know,I don't know,...,NaN,NaN,False,46,m,United States of America,Nebraska,White,United States of America,Nebraska


In [7]:
#Need Columns only 
MentalHealthClean2 = MH_Clean[["Age", "Sex", "Location", "Emp. Coverage","Ranking", "Do you *currently* have a mental health disorder?"]]
print(MentalHealthClean2)

     Age           Sex   Location Emp. Coverage  Ranking  \
0     25          Male   Nebraska  I don't know        4   
1     51          male   Nebraska           Yes        1   
2     27          Male   Illinois  I don't know        2   
3     37          male   Nebraska  I don't know        3   
4     46             m   Nebraska  I don't know        3   
..   ...           ...        ...           ...      ...   
347   27          male        NaN  I don't know        3   
348   48             m  Louisiana           NaN        3   
349   50             M        NaN           NaN        5   
350   30        female        NaN            No        2   
351   24  Female (cis)     Oregon           Yes        3   

    Do you *currently* have a mental health disorder?  
0                                          Don't Know  
1                                            Possibly  
2                                                  No  
3                                                  No  

In [8]:
# Unique labels 
MentalHealthClean2['Sex'].unique()

array(['Male', 'male', 'm', 'female', 'Female', nan, 'M', 'F', 'f',
       'Let\'s keep it simple and say "male"', 'Non-binary',
       'Identify as male', 'Male ', 'Woman', 'Non binary', 'Masculine',
       'Cishet male', 'None', 'Female-identified', 'woman', 'Nonbinary',
       'agender', 'Man', 'Questioning', 'cis male', 'Cis Male',
       'I have a penis', 'rr', 'cis woman', 'Agender trans woman',
       'Female ', 'femmina', '43', 'Trans man', 'man', 'masculino',
       'I am a Wookie', 'Make', 'Trans non-binary/genderfluid',
       'CIS Male', 'Non-binary and gender fluid', 'Femile',
       'Female (cis)'], dtype=object)

In [9]:
#Change male leaning
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['m'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['M'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Let\'s keep it simple and say "male"'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Masculine'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Cishet male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Man'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['cis male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Trans Man'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['I have a penis'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['man'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['masculino'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['CIS Male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Cis Male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Cis Male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Cis Male'],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Male '],'Male')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Trans man'],'Male')

/Users/heatherallardice/opt/anaconda3/envs/Pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/heatherallardice/opt/anaconda3/envs/Pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/heatherallardice/opt/anaconda3/envs/Pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWar

In [10]:
#Change female leaning
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['f'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['F'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Woman'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Female-identified'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['woman'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Agender trans woman'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Female (cis)'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Femile'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['femmina'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['female'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['cis woman'],'Female')
MentalHealthClean2['Sex'] = MentalHealthClean2['Sex'].replace(['Female '],'Female')


/Users/heatherallardice/opt/anaconda3/envs/Pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/heatherallardice/opt/anaconda3/envs/Pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/heatherallardice/opt/anaconda3/envs/Pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWar

In [11]:
MentalHealthClean2['Sex'].unique()

array(['Male', 'Female', nan, 'Non-binary', 'Identify as male',
       'Non binary', 'None', 'Nonbinary', 'agender', 'Questioning', 'rr',
       '43', 'I am a Wookie', 'Make', 'Trans non-binary/genderfluid',
       'Non-binary and gender fluid'], dtype=object)

In [12]:
# Number by gender
MentalHealthClean2['Sex']

0        Male
1        Male
2        Male
3        Male
4        Male
        ...  
347      Male
348      Male
349      Male
350    Female
351    Female
Name: Sex, Length: 352, dtype: object

In [13]:
#Dropping other labels 
L= ['Non-binary', 'Non binary', 'None',
       'Nonbinary', 'agender', 'Questioning', 'rr', '43', 'I am a Wookie',
       'Make', 'Trans non-binary/genderfluid',
       'Non-binary and gender fluid']

print (MentalHealthClean2[~MentalHealthClean2.Sex.isin(L)])
MentalHealthClean2.head()

     Age     Sex   Location Emp. Coverage  Ranking  \
0     25    Male   Nebraska  I don't know        4   
1     51    Male   Nebraska           Yes        1   
2     27    Male   Illinois  I don't know        2   
3     37    Male   Nebraska  I don't know        3   
4     46    Male   Nebraska  I don't know        3   
..   ...     ...        ...           ...      ...   
347   27    Male        NaN  I don't know        3   
348   48    Male  Louisiana           NaN        3   
349   50    Male        NaN           NaN        5   
350   30  Female        NaN            No        2   
351   24  Female     Oregon           Yes        3   

    Do you *currently* have a mental health disorder?  
0                                          Don't Know  
1                                            Possibly  
2                                                  No  
3                                                  No  
4                                                  No  
..             

,Age,Sex,Location,Emp. Coverage,Ranking,Do you *currently* have a mental health disorder?
0,25,Male,Nebraska,I don't know,4,Don't Know
1,51,Male,Nebraska,Yes,1,Possibly
2,27,Male,Illinois,I don't know,2,No
3,37,Male,Nebraska,I don't know,3,No
4,46,Male,Nebraska,I don't know,3,No


In [14]:
output_data_file = "MentalH.csv"

In [15]:
#split data frame by sex
# This is so that we can join by location more easily
#NaN were not removed from the osmi data set because ever row contained at least one NaN

Female_osmi_data =MentalHealthClean2 [MentalHealthClean2['Sex'] == 'Female'] 
Male_osmi_data=MentalHealthClean2[MentalHealthClean2['Sex'] == 'Male']


In [16]:
#check dataframe 

Female_osmi_data.head()
Male_osmi_data.head()

,Age,Sex,Location,Emp. Coverage,Ranking,Do you *currently* have a mental health disorder?
0,25,Male,Nebraska,I don't know,4,Don't Know
1,51,Male,Nebraska,Yes,1,Possibly
2,27,Male,Illinois,I don't know,2,No
3,37,Male,Nebraska,I don't know,3,No
4,46,Male,Nebraska,I don't know,3,No


In [17]:
#save split datasets
Female_osmi_data.to_csv('Female_osmi_data.csv', index = False, header=True)
Male_osmi_data.to_csv('Male_osmi_data.csv', index = False, header=True)

In [18]:
#Join male data sets and female datasets
males_merge= pd.merge(Male_osmi_data, Males, on='Location')
males_merge=males_merge.drop(columns=["Sex_y"])


Females_merge= pd.merge(Female_osmi_data, Females, on='Location')
Females_merge=Females_merge.drop(columns=["Sex_y"])

Females_merge.head()

,Age,Sex_x,Location,Emp. Coverage,Ranking,Do you *currently* have a mental health disorder?,Bad Days Count,Bad Days Mean,Bad Days Min,Bad Days Q1,Bad Days Q2,Bad Days Q3,Bad Days Max
0,36,Female,Nebraska,Yes,3,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
1,39,Female,Nebraska,Yes,3,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
2,20,Female,Nebraska,Yes,2,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
3,19,Female,Nebraska,Not eligible for coverage / NA,3,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
4,26,Female,Pennsylvania,Yes,3,Yes,1,5.2,5.2,5.2,5.2,5.2,5.2


In [19]:
#making the master sheet by cocatinating the tow joins. This will add the sheets together without dropping any rows
Master_sheet= pd.concat([Females_merge, males_merge], ignore_index=True)
Master_sheet.head(100)

,Age,Sex_x,Location,Emp. Coverage,Ranking,Do you *currently* have a mental health disorder?,Bad Days Count,Bad Days Mean,Bad Days Min,Bad Days Q1,Bad Days Q2,Bad Days Q3,Bad Days Max
0,36,Female,Nebraska,Yes,3,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
1,39,Female,Nebraska,Yes,3,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
2,20,Female,Nebraska,Yes,2,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
3,19,Female,Nebraska,Not eligible for coverage / NA,3,Yes,1,4.5,4.5,4.5,4.5,4.5,4.5
4,26,Female,Pennsylvania,Yes,3,Yes,1,5.2,5.2,5.2,5.2,5.2,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,63,Male,Georgia,Yes,3,No,1,4.3,4.3,4.3,4.3,4.3,4.3
96,35,Male,North Carolina,Yes,2,Possibly,1,3.6,3.6,3.6,3.6,3.6,3.6
97,35,Male,North Carolina,NaN,2,Possibly,1,3.6,3.6,3.6,3.6,3.6,3.6
98,30,Male,North Carolina,I don't know,3,No,1,3.6,3.6,3.6,3.6,3.6,3.6


In [20]:
#wrirte master to csv
Master_sheet.to_csv('Master_sheet.csv', index = False, header=True)

In [244]:
MH_Clean['Do you *currently* have a mental health disorder?'].unique()

array(["Don't Know", 'Possibly', 'No', 'Yes'], dtype=object)

In [245]:
# DB connectivity
client = MongoClient('localhost', 27017)
db = client.db
collection = db.collection
# Function to parse csv to dictionary
def csv_to_dict():
    reader = csv.DictReader(open(FILEPATH))
    result = {}
    for row in reader:
        key = row.pop('First_value')
        result[key] = row
    return query
# Final insert statement
db.collection.insert_one(csv_to_dict())

NameError: name 'FILEPATH' is not defined

In [227]:
#Dropping other labels 
L= ["Don't Know", 'Possibly']


print (MentalHealthClean2[~MentalHealthClean2."Do you *currently* have a mental health disorder?".isin(L)])
MentalHealthClean2

SyntaxError: invalid syntax (<ipython-input-227-7e4af903ef01>, line 5)